#必做项

搭载Google Drive

In [8]:
from google.colab import drive
drive.mount("/content/drive")

import os
path="/content/drive/MyDrive/NeuralRecon-W-test"
os.chdir(path)
os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['metrics.py',
 'README.md',
 'LICENSE',
 'requirements.txt',
 'losses.py',
 'environment.yaml',
 'tools',
 'scripts',
 'utils',
 'models',
 'evaluations',
 'assets',
 'rendering',
 'opt.py',
 'train.py',
 'lightning_modules',
 'datasets',
 '.gitignore',
 'data',
 'weights',
 'log',
 'config']

# 第一次初始化操作

操作说明与仓库地址：https://github.com/zju3dv/NeuralRecon-W

下载数据，示例见：（已完成）

```shell
mkdir data && cd data
gdown --id 1eZvmk4GQkrRKUNZpagZEIY_z8Lsdw94v
```

此处参见unzipfile.ipynb

对所有scene生成Ray Cache，例如：

```python
for SCENE_NAME in brandenburg_gate lincoln_memorial palacio_de_bellas_artes pantheon_exterior; do
  scripts/data_generation.sh data/heritage-recon/${SCENE_NAME}
done
```



In [10]:
!ls

assets	  environment.yaml   log	 opt.py		   scripts   weights
config	  evaluations	     losses.py	 README.md	   tools
data	  LICENSE	     metrics.py  rendering	   train.py
datasets  lightning_modules  models	 requirements.txt  utils


本来应用于生成ray cache数据，目前kaolin一直报错
提了一个Issue：https://github.com/NVIDIAGameWorks/kaolin/issues/597

In [11]:
!sh scripts/data_generation.sh data/heritage-recon/brandenburg_gate


+ set -u
+ set -e
+ date +%Y%m%d_%H%M%S
+ now=20220727_083244
+ pwd
+ echo working directory is /content/drive/MyDrive/NeuralRecon-W-test
working directory is /content/drive/MyDrive/NeuralRecon-W-test
+ jobname=data-generation-data/heritage-recon/brandenburg_gate-20220727_083244
+ export CUDA_VISIBLE_DEVICES=5
+ dataset_name=phototourism
+ cache_dir=cache_sgs
+ root_dir=data/heritage-recon/brandenburg_gate
+ min_observation=-1
+ [ ! -f data/heritage-recon/brandenburg_gate/brandenburg_gate.tsv ]
+ python tools/prepare_data/prepare_data_cache.py --root_dir data/heritage-recon/brandenburg_gate --dataset_name phototourism --cache_dir cache_sgs --img_downscale 1 --semantic_map_path semantic_maps --split_to_chunks 64
+ tee log/data-generation-data/heritage-recon/brandenburg_gate-20220727_083244.log
Traceback (most recent call last):
  File "tools/prepare_data/prepare_data_cache.py", line 5, in <module>
    from datasets import dataset_dict
  File "./datasets/__init__.py", line 1, in <module>

In [12]:
!sh scripts/data_generation.sh data/heritage-recon/lincoln_memorial
!shscripts/data_generation.sh data/heritage-recon/palacio_de_bellas_artes
!sh scripts/data_generation.sh data/heritage-recon/pantheon_exterior

+ set -u
+ set -e
+ date +%Y%m%d_%H%M%S
+ now=20220727_083257
+ pwd
+ echo working directory is /content/drive/MyDrive/NeuralRecon-W-test
working directory is /content/drive/MyDrive/NeuralRecon-W-test
+ jobname=data-generation-data/heritage-recon/lincoln_memorial-20220727_083257
+ export CUDA_VISIBLE_DEVICES=5
+ dataset_name=phototourism
+ cache_dir=cache_sgs
+ root_dir=data/heritage-recon/lincoln_memorial
+ min_observation=-1
+ [ ! -f data/heritage-recon/lincoln_memorial/lincoln_memorial.tsv ]
+ python tools/prepare_data/prepare_data_cache.py --root_dir data/heritage-recon/lincoln_memorial --dataset_name phototourism --cache_dir cache_sgs --img_downscale 1 --semantic_map_path semantic_maps --split_to_chunks 64
+ tee log/data-generation-data/heritage-recon/lincoln_memorial-20220727_083257.log
Traceback (most recent call last):
  File "tools/prepare_data/prepare_data_cache.py", line 5, in <module>
    from datasets import dataset_dict
  File "./datasets/__init__.py", line 1, in <module>

补充requirements包，上一步之前先如下安装

In [ ]:
!pip install torchvision
!pip install kaolin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 日常训练

查看GPU信息

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jul 27 07:58:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

安装依赖库

In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 924 kB 14.8 MB/s 
     |████████████████████████████████| 396 kB 70.3 MB/s 
     |████████████████████████████████| 225 kB 74.5 MB/s 
     |████████████████████████████████| 50.4 MB 1.2 MB/s 
     |████████████████████████████████| 11.6 MB 50.7 MB/s 
     |████████████████████████████████| 3.3 MB 63.9 MB/s 
     |████████████████████████████████| 26.9 MB 28.0 MB/s 
     |████████████████████████████████| 9.5 MB 72.2 MB/s 
     |████████████████████████████████| 61 kB 591 kB/s 
     |████████████████████████████████| 53 kB 2.8 MB/s 
     |████████████████████████████████| 58 kB 6.3 MB/s 
     |████████████████████████████████| 13.2 MB 58.8 MB/s 
     |████████████████████████████████| 265 kB 92.4 MB/s 
     |████████████████████████████████| 105 kB 86.9 MB/s 
     |████████████████████████████████| 628 kB 66.9 MB/s 
     |████████████████████████████